# Train Val Test split analysis
Given a reference dataset containing all available data, how should we split it into train, validation and test set ?

## Reference Dataset statistics 

In [325]:
import os
ROOT_FOLDER = "C:/Users/Fred/Downloads"
DATASET_FOLDER = os.path.join(ROOT_FOLDER,"Glomerulus Data Clean/Reference")

In [328]:
import pandas as pd
dataset = pd.DataFrame(columns = ['marker','patient_number','image_id','roi_id'])
i = 0
for marker in os.listdir(DATASET_FOLDER):
    marker_folder = os.path.join(DATASET_FOLDER,marker)
    for image in os.listdir(marker_folder):
        image_folder = os.path.join(marker_folder,image)
        roi_folder = os.path.join(image_folder,'rois')
        for roi in os.listdir(roi_folder):
            if roi.endswith('.roi'):
                dataset.loc[i] = [marker,image[0:3],image,roi]
                i += 1

In [383]:
dataset.head()

,marker,patient_number,image_id,roi_id
0,IgG,R22,R22 RD IgG_15.0xC1,R22 RD IgG_15.0xC1_0141-0226.roi
1,IgG,R22,R22 RD IgG_15.0xC1,R22 RD IgG_15.0xC1_0155-0887.roi
2,IgG,R22,R22 RD IgG_15.0xC1,R22 RD IgG_15.0xC1_0269-0517.roi
3,IgG,R22,R22 RD IgG_15.0xC1,R22 RD IgG_15.0xC1_0306-1824.roi
4,IgG,R22,R22 RD IgG_15.0xC1,R22 RD IgG_15.0xC1_0377-0146.roi


In [330]:
dataset.to_csv(os.path.join(ROOT_FOLDER,'dataset.csv'))

In [385]:
import numpy as np 

def stats(df): # group data in df by marker, in value and in percentage
    dfs = df.groupby(['marker']).nunique().drop('marker',axis=1).reset_index()
    dfs.loc[len(dfs)]=['TOTAL'] + dfs[dfs.columns[1:]].sum().tolist()
    dfs.columns=['marker','nb_patients','nb_images','nb_glomerulus']
    add_col = ['%patients','%images','%glomerulus']
    for colnum, col in enumerate(dfs.columns[1:]):
        dfs[add_col[colnum]] = np.round(dfs[col]/dfs.iloc[len(dfs)-1,colnum+1]*100,1)  
    return dfs

In [386]:
dfg = stats(dataset)
dfg

,marker,nb_patients,nb_images,nb_glomerulus,%patients,%images,%glomerulus
0,IgG,22,130,922,25.3,22.6,25.3
1,PAS,32,234,1211,36.8,40.8,33.2
2,VEGF,33,210,1510,37.9,36.6,41.4
3,TOTAL,87,574,3643,100.0,100.0,100.0


## Train - Val - Test strategy : 
We aim for a 80 / 10 / 10 split in terms of glomerulus numbers   
All sets should have the same percentage of glomerulus per marker  
NB : For the same patient we may have samples with two different markers    
=> To avoid data leakage, we need to split data at the patient level  

We've generated all possible selection for validation and test set with a tolerance of 9.5 to 10.5 of glomerulus total



In [345]:
markers = dataset['marker'].unique() # array(['IgG', 'PAS', 'VEGF'], dtype=object)
pool = dataset['patient_number'].unique().tolist()

def count_glomerulus(v_selection): #computes the count of glomerulus we have on a selection of patient
    return dataset.loc[dataset['patient_number'].isin(v_selection),'roi_id'].count()

def count_marker_glomerulus(v_selection): #computes the count of glomerulus per marker we have on a selection of patient
    counts = []
    for marker in markers: 
        counts.append(dataset.loc[(dataset['patient_number'].isin(v_selection)) & (dataset['marker']==marker),'roi_id'].count())
    return counts    

In [350]:
total_glomerulus = dataset['roi_id'].count()
nbg_max = int(10.5/100 * total_glomerulus)
nbg_min = int(9.5/100 * total_glomerulus)
print(f'target nb of glomerulus in the set : min = {nbg_min}, max = {nbg_max}')

target nb of glomerulus in the set : min = 346, max = 382


In [314]:
def save(row,array): # appends row to array
    if (row.size == 0):
        return array
    else:
        row = row.reshape(-1,5)
        if (array.size == 0):
            return row
        else:
            return np.append(array,row,axis=0)

def score(v_selection, v_pool):

    score_array= np.array([])
    if not v_pool: # pool vide
        return np.array([])

    for patient in v_pool:
#         print('selection courante:',v_selection)
#         print('patient supplémentaire sélectionné:', patient)
        nbg = count_glomerulus(v_selection+[patient])
#         print('nbg',nbg)

        if nbg > nbg_max: #too much glomerulus 
#             print('>nbg_max')
            return np.array([])
#             print('this line should not print')
        elif nbg >= nbg_min : #within tolerance -> save this selection
#             print('>=nbg_min')
            # ligne à ajouter au tableau de score : la selection et son décompte de glomérules
            new_line = np.array([v_selection+[patient]] 
                    + count_marker_glomerulus(v_selection+[patient])) 
            new_line = new_line.reshape(1,-1)
            print(new_line)
#             print(f'appending {new_line} to {score_array}')
            score_array = save(new_line,score_array)
#             print('resultat :',score_array)
        # in any case (other than nbg > nbg_max), try adding another patient
#         print('trying to add another patient')
        new_pool = v_pool.copy()
        index = v_pool.index(patient)+1 
        new_pool = new_pool[index:]
#         print('newpool',new_pool)
        new_line = score(v_selection + [patient], new_pool)
#         print(f'appending2 {new_line} to {score_array}')
        score_array = save(new_line,score_array)
    return score_array

In [316]:
solutions = score([],pool)

[[list(['R53', 'R57', 'R58']) 91 101 112 102]]
[[list(['R53', 'R22', 'R27']) 31 152 40 145]]
[[list(['R53', 'R29', 'R30']) 31 109 134 115]]
[[list(['R53', 'R29', 'R32']) 31 107 128 116]]
[[list(['R53', 'R29', 'R37']) 31 115 128 113]]
[[list(['R53', 'R29', 'R38']) 31 118 127 117]]
[[list(['R53', 'R29', 'R41']) 31 117 128 121]]
[[list(['R53', 'R30', 'R32']) 31 113 116 117]]
[[list(['R53', 'R30', 'R37']) 31 121 116 114]]
[[list(['R53', 'R30', 'R38']) 31 124 115 118]]
[[list(['R53', 'R30', 'R41']) 31 123 116 122]]
[[list(['R53', 'R30', 'R43']) 31 116 120 135]]
[[list(['R53', 'R32', 'R37']) 31 119 110 115]]
[[list(['R53', 'R32', 'R38']) 31 122 109 119]]
[[list(['R53', 'R32', 'R41']) 31 121 110 123]]
[[list(['R53', 'R32', 'R43']) 31 114 114 136]]
[[list(['R53', 'R37', 'R38']) 31 130 109 116]]
[[list(['R53', 'R37', 'R41']) 31 129 110 120]]
[[list(['R53', 'R37', 'R43']) 31 122 114 133]]
[[list(['R53', 'R38', 'R41']) 31 132 109 124]]
[[list(['R53', 'R38', 'R43']) 31 125 113 137]]
[[list(['R53',

[[list(['R54', 'R58', 'R15', 'R21', 'R44']) 56 50 174 103]]
[[list(['R54', 'R58', 'R15', 'R21', 'R45']) 56 50 178 95]]
[[list(['R54', 'R58', 'R15', 'R21', 'R46']) 56 50 179 100]]
[[list(['R54', 'R58', 'R15', 'R21', 'R47']) 56 50 176 105]]
[[list(['R54', 'R58', 'R15', 'R21', 'R48']) 56 50 175 93]]
[[list(['R54', 'R58', 'R15', 'R21', 'R50']) 56 50 179 89]]
[[list(['R54', 'R58', 'R15', 'R21', 'R51']) 56 50 178 102]]
[[list(['R54', 'R58', 'R15', 'R51', 'R59']) 56 50 174 102]]
[[list(['R54', 'R58', 'R21', 'R33']) 56 50 153 119]]
[[list(['R54', 'R58', 'R21', 'R34']) 56 50 157 107]]
[[list(['R54', 'R58', 'R21', 'R35']) 56 50 151 110]]
[[list(['R54', 'R58', 'R21', 'R51', 'R59']) 56 50 189 102]]
[[list(['R54', 'R58', 'R31', 'R33']) 56 50 153 145]]
[[list(['R54', 'R58', 'R31', 'R34']) 56 50 157 133]]
[[list(['R54', 'R58', 'R31', 'R35']) 56 50 151 136]]
[[list(['R54', 'R58', 'R31', 'R42']) 56 50 153 123]]
[[list(['R54', 'R58', 'R31', 'R44']) 56 50 147 129]]
[[list(['R54', 'R58', 'R31', 'R45']) 56

[[list(['R54', 'R39', 'R48', 'R51']) 22 80 110 174]]
[[list(['R54', 'R39', 'R50', 'R51']) 22 80 114 170]]
[[list(['R54', 'R40', 'R52']) 22 145 73 141]]
[[list(['R54', 'R40', 'R15', 'R33']) 22 89 103 153]]
[[list(['R54', 'R40', 'R15', 'R51', 'R59']) 22 89 139 136]]
[[list(['R54', 'R40', 'R21', 'R33']) 22 89 118 153]]
[[list(['R54', 'R40', 'R21', 'R34']) 22 89 122 141]]
[[list(['R54', 'R40', 'R21', 'R35']) 22 89 116 144]]
[[list(['R54', 'R40', 'R21', 'R51', 'R59']) 22 89 154 136]]
[[list(['R54', 'R40', 'R44', 'R45']) 22 89 109 183]]
[[list(['R54', 'R40', 'R45', 'R46']) 22 89 114 180]]
[[list(['R54', 'R40', 'R47', 'R48']) 22 89 108 183]]
[[list(['R54', 'R40', 'R47', 'R50']) 22 89 112 179]]
[[list(['R54', 'R40', 'R48', 'R50']) 22 89 111 167]]
[[list(['R54', 'R40', 'R48', 'R51']) 22 89 110 180]]
[[list(['R54', 'R40', 'R50', 'R51']) 22 89 114 176]]
[[list(['R54', 'R41', 'R43']) 22 117 109 136]]
[[list(['R54', 'R52', 'R15', 'R21']) 22 106 142 107]]
[[list(['R54', 'R52', 'R15', 'R31']) 22 106 

[[list(['R54', 'R47', 'R51', 'R59']) 22 50 149 158]]
[[list(['R55', 'R57', 'R58']) 91 94 109 98]]
[[list(['R55', 'R22', 'R24', 'R25']) 31 177 37 158]]
[[list(['R55', 'R29', 'R30']) 31 102 131 111]]
[[list(['R55', 'R29', 'R32']) 31 100 125 112]]
[[list(['R55', 'R29', 'R37']) 31 108 125 109]]
[[list(['R55', 'R29', 'R38']) 31 111 124 113]]
[[list(['R55', 'R29', 'R39', 'R40']) 31 142 90 142]]
[[list(['R55', 'R29', 'R41']) 31 110 125 117]]
[[list(['R55', 'R29', 'R43']) 31 103 129 130]]
[[list(['R55', 'R30', 'R37']) 31 114 113 110]]
[[list(['R55', 'R30', 'R38']) 31 117 112 114]]
[[list(['R55', 'R30', 'R39', 'R40']) 31 148 78 143]]
[[list(['R55', 'R30', 'R41']) 31 116 113 118]]
[[list(['R55', 'R30', 'R43']) 31 109 117 131]]
[[list(['R55', 'R32', 'R38']) 31 115 106 115]]
[[list(['R55', 'R32', 'R39', 'R40']) 31 146 72 144]]
[[list(['R55', 'R32', 'R41']) 31 114 107 119]]
[[list(['R55', 'R32', 'R43']) 31 107 111 132]]
[[list(['R55', 'R37', 'R38']) 31 123 106 112]]
[[list(['R55', 'R37', 'R39', 'R4

[[list(['R55', 'R21', 'R46', 'R47']) 31 50 156 157]]
[[list(['R55', 'R21', 'R46', 'R48']) 31 50 155 145]]
[[list(['R55', 'R21', 'R46', 'R50']) 31 50 159 141]]
[[list(['R55', 'R21', 'R46', 'R51']) 31 50 158 154]]
[[list(['R55', 'R21', 'R47', 'R48']) 31 50 152 150]]
[[list(['R55', 'R21', 'R47', 'R50']) 31 50 156 146]]
[[list(['R55', 'R21', 'R47', 'R51']) 31 50 155 159]]
[[list(['R55', 'R21', 'R48', 'R50']) 31 50 155 134]]
[[list(['R55', 'R21', 'R48', 'R51']) 31 50 154 147]]
[[list(['R55', 'R21', 'R50', 'R51']) 31 50 158 143]]
[[list(['R55', 'R31', 'R48', 'R50']) 31 50 155 160]]
[[list(['R55', 'R33', 'R34']) 31 50 123 178]]
[[list(['R55', 'R33', 'R35']) 31 50 117 181]]
[[list(['R55', 'R33', 'R42']) 31 50 119 168]]
[[list(['R55', 'R33', 'R44']) 31 50 113 174]]
[[list(['R55', 'R33', 'R46']) 31 50 118 171]]
[[list(['R55', 'R33', 'R47']) 31 50 115 176]]
[[list(['R55', 'R33', 'R51']) 31 50 117 173]]
[[list(['R55', 'R34', 'R35']) 31 50 121 169]]
[[list(['R55', 'R34', 'R51', 'R59']) 31 50 159 16

...

[[list(['R15', 'R33', 'R44', 'R48', 'R51']) 0 0 178 221]]
[[list(['R15', 'R33', 'R44', 'R50', 'R51']) 0 0 182 217]]
[[list(['R15', 'R33', 'R45', 'R47', 'R48']) 0 0 180 216]]
[[list(['R15', 'R33', 'R45', 'R47', 'R50']) 0 0 184 212]]
[[list(['R15', 'R33', 'R45', 'R48', 'R50']) 0 0 183 200]]
[[list(['R15', 'R33', 'R45', 'R48', 'R51']) 0 0 182 213]]
[[list(['R15', 'R33', 'R45', 'R50', 'R51']) 0 0 186 209]]
[[list(['R15', 'R33', 'R46', 'R47', 'R48']) 0 0 181 221]]
[[list(['R15', 'R33', 'R46', 'R47', 'R50']) 0 0 185 217]]
[[list(['R15', 'R33', 'R46', 'R48', 'R50']) 0 0 184 205]]
[[list(['R15', 'R33', 'R46', 'R48', 'R51']) 0 0 183 218]]
[[list(['R15', 'R33', 'R46', 'R50', 'R51']) 0 0 187 214]]
[[list(['R15', 'R33', 'R47', 'R48', 'R50']) 0 0 181 210]]
[[list(['R15', 'R33', 'R47', 'R48', 'R51']) 0 0 180 223]]
[[list(['R15', 'R33', 'R47', 'R50', 'R51']) 0 0 184 219]]
[[list(['R15', 'R33', 'R48', 'R50', 'R51']) 0 0 183 207]]
[[list(['R15', 'R34', 'R35', 'R44', 'R45']) 0 0 185 219]]
[[list(['R15',

[[list(['R15', 'R42', 'R45', 'R47', 'R51']) 0 0 183 203]]
[[list(['R15', 'R42', 'R45', 'R48', 'R51']) 0 0 182 191]]
[[list(['R15', 'R42', 'R45', 'R50', 'R51']) 0 0 186 187]]
[[list(['R15', 'R42', 'R46', 'R47', 'R48']) 0 0 181 199]]
[[list(['R15', 'R42', 'R46', 'R47', 'R50']) 0 0 185 195]]
[[list(['R15', 'R42', 'R46', 'R47', 'R51']) 0 0 184 208]]
[[list(['R15', 'R42', 'R46', 'R48', 'R50']) 0 0 184 183]]
[[list(['R15', 'R42', 'R46', 'R48', 'R51']) 0 0 183 196]]
[[list(['R15', 'R42', 'R46', 'R50', 'R51']) 0 0 187 192]]
[[list(['R15', 'R42', 'R47', 'R48', 'R50']) 0 0 181 188]]
[[list(['R15', 'R42', 'R47', 'R48', 'R51']) 0 0 180 201]]
[[list(['R15', 'R42', 'R47', 'R50', 'R51']) 0 0 184 197]]
[[list(['R15', 'R42', 'R48', 'R50', 'R51']) 0 0 183 185]]
[[list(['R15', 'R42', 'R48', 'R50', 'R51', 'R59']) 0 0 221 185]]
[[list(['R15', 'R44', 'R45', 'R46', 'R47']) 0 0 178 207]]
[[list(['R15', 'R44', 'R45', 'R46', 'R48']) 0 0 177 195]]
[[list(['R15', 'R44', 'R45', 'R46', 'R50']) 0 0 181 191]]
[[list(

[[list(['R21', 'R31', 'R42', 'R47', 'R50']) 0 0 202 170]]
[[list(['R21', 'R31', 'R42', 'R47', 'R51']) 0 0 201 183]]
[[list(['R21', 'R31', 'R42', 'R48', 'R51']) 0 0 200 171]]
[[list(['R21', 'R31', 'R42', 'R50', 'R51']) 0 0 204 167]]
[[list(['R21', 'R31', 'R44', 'R45', 'R46']) 0 0 198 177]]
[[list(['R21', 'R31', 'R44', 'R45', 'R47']) 0 0 195 182]]
[[list(['R21', 'R31', 'R44', 'R45', 'R51']) 0 0 197 179]]
[[list(['R21', 'R31', 'R44', 'R46', 'R47']) 0 0 196 187]]
[[list(['R21', 'R31', 'R44', 'R46', 'R48']) 0 0 195 175]]
[[list(['R21', 'R31', 'R44', 'R46', 'R50']) 0 0 199 171]]
[[list(['R21', 'R31', 'R44', 'R46', 'R51']) 0 0 198 184]]
[[list(['R21', 'R31', 'R44', 'R47', 'R48']) 0 0 192 180]]
[[list(['R21', 'R31', 'R44', 'R47', 'R50']) 0 0 196 176]]
[[list(['R21', 'R31', 'R44', 'R47', 'R51']) 0 0 195 189]]
[[list(['R21', 'R31', 'R44', 'R48', 'R51']) 0 0 194 177]]
[[list(['R21', 'R31', 'R44', 'R50', 'R51']) 0 0 198 173]]
[[list(['R21', 'R31', 'R45', 'R46', 'R47']) 0 0 200 179]]
[[list(['R21',

[[list(['R21', 'R45', 'R46', 'R47', 'R51']) 0 0 197 206]]
[[list(['R21', 'R45', 'R46', 'R48', 'R50']) 0 0 197 181]]
[[list(['R21', 'R45', 'R46', 'R48', 'R51']) 0 0 196 194]]
[[list(['R21', 'R45', 'R46', 'R50', 'R51']) 0 0 200 190]]
[[list(['R21', 'R45', 'R47', 'R48', 'R50']) 0 0 194 186]]
[[list(['R21', 'R45', 'R47', 'R48', 'R51']) 0 0 193 199]]
[[list(['R21', 'R45', 'R47', 'R50', 'R51']) 0 0 197 195]]
[[list(['R21', 'R45', 'R48', 'R50', 'R51']) 0 0 196 183]]
[[list(['R21', 'R46', 'R47', 'R48', 'R50']) 0 0 195 191]]
[[list(['R21', 'R46', 'R47', 'R48', 'R51']) 0 0 194 204]]
[[list(['R21', 'R46', 'R47', 'R50', 'R51']) 0 0 198 200]]
[[list(['R21', 'R46', 'R48', 'R50', 'R51']) 0 0 197 188]]
[[list(['R21', 'R47', 'R48', 'R50', 'R51']) 0 0 194 193]]
[[list(['R31', 'R33', 'R34', 'R35']) 0 0 167 215]]
[[list(['R31', 'R33', 'R34', 'R42']) 0 0 169 202]]
[[list(['R31', 'R33', 'R34', 'R44']) 0 0 163 208]]
[[list(['R31', 'R33', 'R34', 'R45']) 0 0 167 200]]
[[list(['R31', 'R33', 'R34', 'R46']) 0 0 1

[[list(['R33', 'R45', 'R46', 'R50']) 0 0 160 207]]
[[list(['R33', 'R45', 'R46', 'R51']) 0 0 159 220]]
[[list(['R33', 'R45', 'R47', 'R48']) 0 0 153 216]]
[[list(['R33', 'R45', 'R47', 'R50']) 0 0 157 212]]
[[list(['R33', 'R45', 'R47', 'R51']) 0 0 156 225]]
[[list(['R33', 'R45', 'R48', 'R51']) 0 0 155 213]]
[[list(['R33', 'R45', 'R48', 'R51', 'R59']) 0 0 193 213]]
[[list(['R33', 'R45', 'R50', 'R51']) 0 0 159 209]]
[[list(['R33', 'R45', 'R50', 'R51', 'R59']) 0 0 197 209]]
[[list(['R33', 'R46', 'R47', 'R48']) 0 0 154 221]]
[[list(['R33', 'R46', 'R47', 'R50']) 0 0 158 217]]
[[list(['R33', 'R46', 'R47', 'R51']) 0 0 157 230]]
[[list(['R33', 'R46', 'R48', 'R51']) 0 0 156 218]]
[[list(['R33', 'R46', 'R50', 'R51']) 0 0 160 214]]
[[list(['R33', 'R47', 'R48', 'R51']) 0 0 153 223]]
[[list(['R33', 'R47', 'R50', 'R51']) 0 0 157 219]]
[[list(['R33', 'R48', 'R50', 'R51', 'R59']) 0 0 194 207]]
[[list(['R34', 'R35', 'R42', 'R44']) 0 0 160 221]]
[[list(['R34', 'R35', 'R42', 'R45']) 0 0 164 213]]
[[list(['R

After close look we chose : 
* valid : R21, R24, R33, R56 => 9.85% of glomerulus total
* test  : R34, R39, R52, R58 => 9.55% of glomerulus total

Statistics below show a very similar balance of marker type in train, valid and test sets 

In [390]:
validation_set = dataset[dataset['patient_number'].isin(['R21','R24','R33','R56'])]
test_set = dataset[dataset['patient_number'].isin(['R34','R39','R52','R58'])]

In [389]:
stats(dataset)

,marker,nb_patients,nb_images,nb_glomerulus,%patients,%images,%glomerulus
0,IgG,22,130,922,25.3,22.6,25.3
1,PAS,32,234,1211,36.8,40.8,33.2
2,VEGF,33,210,1510,37.9,36.6,41.4
3,TOTAL,87,574,3643,100.0,100.0,100.0


In [388]:
stats(validation_set)

,marker,nb_patients,nb_images,nb_glomerulus,%patients,%images,%glomerulus
0,IgG,2,12,89,25.0,20.7,24.8
1,PAS,3,25,121,37.5,43.1,33.7
2,VEGF,3,21,149,37.5,36.2,41.5
3,TOTAL,8,58,359,100.0,100.0,100.0


In [392]:
stats(test_set)

,marker,nb_patients,nb_images,nb_glomerulus,%patients,%images,%glomerulus
0,IgG,2,12,86,25.0,21.8,24.7
1,PAS,3,23,118,37.5,41.8,33.9
2,VEGF,3,20,144,37.5,36.4,41.4
3,TOTAL,8,55,348,100.0,100.0,100.0
